In [1]:
import pandas as pd

In [2]:
raw_df = pd.DataFrame({
    'state': [
        'WA', 'NY',
        'WA', 'NY',
        'WA',        
        'WA', 'NY',
        'WA', 'NY',
        'WA', 'NY'
    ],
    'date': [
        '12-19-2019', '12-19-2019',
        '12-18-2019', '12-18-2019',
        '12-17-2019',       
        '12-15-2019', '12-15-2019',
        '12-15-2019', '12-15-2019',
        '12-14-2019', '12-14-2019'],    
    'rank': [
        None, None,
        5, 7,
        4,        
        None, None,
        1, 2,
        3, None],
    'x': [
        None, None,
        'a', 'b',
        'c',         
        None, None,
        'e', 'f',
        'g', None]    
})
raw_df.sort_values(by=['state', 'date'])

,date,rank,state,x
10,12-14-2019,NaN,NY,None
6,12-15-2019,NaN,NY,None
8,12-15-2019,2.0,NY,f
3,12-18-2019,7.0,NY,b
1,12-19-2019,NaN,NY,None
9,12-14-2019,3.0,WA,g
5,12-15-2019,NaN,WA,None
7,12-15-2019,1.0,WA,e
4,12-17-2019,4.0,WA,c
2,12-18-2019,5.0,WA,a


In [13]:
raw_df['date'] = pd.to_datetime(raw_df['date'])
daily_df = raw_df.groupby(['state', 'date']).agg({'rank':'mean', 'x':'last'})
daily_df

x  rank
state date                  
NY    2019-12-14  None   NaN
      2019-12-15     f   2.0
      2019-12-18     b   7.0
      2019-12-19  None   NaN
WA    2019-12-14     g   3.0
      2019-12-15     e   1.0
      2019-12-17     c   4.0
      2019-12-18     a   5.0
      2019-12-19  None   NaN

In [15]:
inpl_rank = daily_df.unstack('state')['rank'].interpolate(method='spline', order=1, axis=0)
inpl_rank

state,NY,WA
date,,
2019-12-14,NaN,3.00
2019-12-15,2.000000,1.00
2019-12-17,5.333333,4.00
2019-12-18,7.000000,5.00
2019-12-19,8.666667,5.35


In [28]:
x = daily_df.reindex(inpl_rank.stack().index.swaplevel())
x['rank_2'] = inpl_rank.stack().swaplevel()
x

,,x,rank,rank_2
state,date,,,
WA,2019-12-14,g,3.0,3.000000
NY,2019-12-15,f,2.0,2.000000
WA,2019-12-15,e,1.0,1.000000
NY,2019-12-17,NaN,NaN,5.333333
WA,2019-12-17,c,4.0,4.000000
NY,2019-12-18,b,7.0,7.000000
WA,2019-12-18,a,5.0,5.000000
NY,2019-12-19,None,NaN,8.666667
WA,2019-12-19,None,NaN,5.350000


In [24]:
inpl_rank.stack()

date        state
2019-12-14  WA       3.000000
2019-12-15  NY       2.000000
            WA       1.000000
2019-12-17  NY       5.333333
            WA       4.000000
2019-12-18  NY       7.000000
            WA       5.000000
2019-12-19  NY       8.666667
            WA       5.350000
dtype: float64